| Method    | Description | SQL equivalents |
| -------- | ------- | ------- |
|`Select` |Transforms each input element with the given lambda expression |SELECT
|`SelectMany` |Transforms each input element, and then flattens and concatenates<br>the resultant subsequences |INNER JOIN,<br>LEFT OUTER JOIN,<br>CROSS JOIN

### Select

| Argument    | Type |
| -------- | ------- |
|Source sequence |`IEnumerable<TSource>`
|Result selector |TSource => TResult or (TSource,int) => TResult

In [ ]:
public static IEnumerable<TResult> Select<TSource,TResult>
(this IEnumerable<TSource> source, Func<TSource,TResult> selector)
{
    foreach (TSource element in source)
        yield return selector (element);
}

With ***Select***, you always get the `same number of elements` that you started with. Each element, however, can be `transformed` in any manner by the lambda function.

In [ ]:
using System.Drawing;

//The following selects the names of all fonts installed on the computer
IEnumerable<string> query = 
    from f in FontFamily.Families
    select f.Name;

IEnumerable<string> lamdaQuery = FontFamily.Families.Select (f => f.Name);

In [ ]:
using System.Drawing;
//Select statements are often used to project into anonymous types:
var query =
from f in FontFamily.Families
select new { f.Name, LineSpacing = f.GetLineSpacing (FontStyle.Bold) };

***Indexed projection***

In [ ]:
string[] names = { "Tom", "Dick", "Harry", "Mary", "Jay" };
IEnumerable<string> query = names
.Select ((s,i) => i + "=" + s); // { "0=Tom", "1=Dick", ... }

***Select subqueries and object hierarchies***

In [ ]:
using System.IO;

string tempPath = Path.GetTempPath();
DirectoryInfo[] dirs = new DirectoryInfo (tempPath).GetDirectories();

var query =
from d in dirs
where (d.Attributes & FileAttributes.System) == 0
select new
{
    DirectoryName = d.FullName,
    Created = d.CreationTime,
    Files = from f in d.GetFiles()
        where (f.Attributes & FileAttributes.Hidden) == 0
        select new { FileName = f.Name, f.Length, }
};

foreach (var dirFiles in query)
{
    Console.WriteLine ("Directory: " + dirFiles.DirectoryName);
    foreach (var file in dirFiles.Files)
    Console.WriteLine (" " + file.FileName + " Len: " + file.Length);
}


***correlated subquery***  
subquery is **correlated** if it `references` an object in the `outer query`

With ***local queries***, a `subquery` `within a Select` causes **double-deferred execution**.

<div dir="rtl" style="width:80%; margin:auto">
هر پوشه تنها زمانی مورد بررسی قرار می‌گیرد که در حلقه بیرونی به آن رسیده شود و فایل‌های هر پوشه نیز فقط زمانی بررسی و فیلتر می‌شوند که به آن‌ها در حلقه داخلی نیاز باشد. این رویکرد منجر به صرفه‌جویی در منابع و افزایش کارایی می‌شود، چرا که تنها داده‌های مورد نیاز در هر لحظه پردازش می‌شوند.

</div>

***Subqueries and joins in EF Core***

In [ ]:
var query =
    from c in dbContext.Customers
    select new {
            c.Name,
            Purchases = (from p in dbContext.Purchases
            where p.CustomerID == c.ID && p.Price > 1000
            select new { p.Description, p.Price })
        .ToList()
};

foreach (var namePurchases in query)
{
    Console.WriteLine ("Customer: " + namePurchases.Name);
    foreach (var purchaseDetail in namePurchases.Purchases)
        Console.WriteLine (" - $$$: " + purchaseDetail.Price);
}

<div dir="rtl" style="width:80%; margin:auto">
<h3>جوین در دیتابیس‌های رابطه‌ای</h3>
<p>در یک پایگاه داده رابطه‌ای معمولی، وقتی از عملیات جوین (Join) استفاده می‌کنیم، دو جدول مختلف را بر اساس یک یا چند شرط مشترک به هم متصل می‌کنیم. نتیجه یک جدول دو بعدی است که در آن هر سطر ممکن است داده‌ها از هر دو جدول را در خود داشته باشد. این جدول نتیجه معمولاً به صورت یک نما (view) یا یک مجموعه داده‌های فلت (flat dataset) است که همه اطلاعات مربوطه در سطرهای متوالی قرار دارند.</p>

<h3>پرس و جو در Entity Framework Core</h3>
<p>در مقابل، Entity Framework Core با استفاده از LINQ این امکان را فراهم می‌کند که به جای ساخت یک جدول دو بعدی فلت، داده‌ها را به صورت سلسله مراتبی سازمان‌دهی کنیم. </p>

<h3>مزایای ساختار سلسله مراتبی</h3>

<p>این رویکرد ساختارمند بودن داده‌ها به توسعه‌دهندگان کمک می‌کند تا به طور طبیعی با داده‌ها به شکلی که بیشتر منعکس‌کننده ساختار شیء‌گرای برنامه است، کار کنند. برخلاف نمایش دو بعدی و فلت، این ساختار به توسعه‌دهندگان اجازه می‌دهد که روابط پیچیده و سلسله مراتبی میان انواع مختلف داده‌ها را به طور مؤثرتری مدیریت کنند.</p>
</div>

In [ ]:
// dont need to use p.CustomerID == c.ID
var query =
    from c in dbContext.Customers
    select new {
            c.Name,
            Purchases = 
            (from p in dbContext.Purchases
            where  p.Price > 1000
            select new { p.Description, p.Price })
        .ToList()
};

In [ ]:
var query =
    from c in dbContext.Customers
    where c.Purchases.Any (p => p.Price > 1000)
    select new {
            c.Name,
            Purchases = 
            (from p in dbContext.Purchases
            where p.Price > 1000
            select new { p.Description, p.Price })
        .ToList()
};

In [ ]:
//we can do above code by below code 
var query =
    from c in dbContext.Customers
    let highValueP = from p in c.Purchases
                    where p.Price > 1000
                    select new { p.Description, p.Price }
    where highValueP.Any()
    select new { c.Name, Purchases = highValueP };

***Projecting into concrete types***

### SelectMany

| Argument    | Type |
| -------- | ------- |
|Source sequence |`IEnumerable<TSource>`
|Result selector |TSource => IEnumerable<TResult> or (TSource,int) => IEnumerable<TResult>

In [ ]:
public static IEnumerable<TResult> SelectMany<TSource,TResult>
(IEnumerable<TSource> source,
Func <TSource,IEnumerable<TResult>> selector)
{
    foreach (TSource element in source)
        foreach (TResult subElement in selector (element))
            yield return subElement;
}

<div dir="rtl" style="width:80%; margin:auto">
<p><code>Select</code> برای انتخاب و تبدیل هر عنصر از یک مجموعه استفاده می‌شود. این تابع به شما امکان می‌دهد برای هر عنصر در مجموعه، یک عملیات تبدیل انجام دهید و یک مجموعه جدید با همان تعداد عنصر اما احتمالا با نوع متفاوت ایجاد کنید. <code>Select</code> به شما این امکان را می‌دهد که از هر عنصر ورودی، یک عنصر خروجی تولید کنید.</p>

</div>

In [ ]:
var numbers = new List<int> { 1, 2, 3 };
var doubled = numbers.Select(x => x * 2);  // { 2, 4, 6 }

<div dir="rtl" style="width:80%; margin:auto">
<p><code>SelectMany</code> برای فلت کردن مجموعه‌های مجموعه‌ای (collections of collections) به کار می‌رود. این تابع هر عنصر از یک مجموعه را می‌گیرد، که خود می‌تواند یک مجموعه باشد، و تمام مجموعه‌های فرعی را به یک مجموعه واحد فلت تبدیل می‌کند. <code>SelectMany</code> بسیار مفید است وقتی که شما می‌خواهید از هر عنصر چندین عنصر خروجی تولید کنید.</p>

<h3>تفاوت‌های کلیدی بین <code>Select</code> و <code>SelectMany</code></h3>
<p><strong>تبدیل versus فلت‌سازی</strong>: <code>Select</code> یک عملیات ۱-به-۱ (one-to-one) بر روی عناصر انجام می‌دهد، در حالی که <code>SelectMany</code> یک عملیات ۱-به-چند (one-to-many) انجام می‌دهد و نتیجه را فلت می‌کند.</p>
</div>

In [ ]:
var listOfLists = new List<List<int>>
{
    new List<int> { 1, 2 },
    new List<int> { 3, 4 },
    new List<int> { 5, 6 }
};
var allNumbers = listOfLists.SelectMany(x => x);  // { 1, 2, 3, 4, 5, 6 }


In [ ]:
public class Person
{
    public int Id { get; set; }

    public string Name { get; set; }

    public List<Purchase> Purchases { get; set; }
}

public class Purchase
{
    public int Id { get; set; }

    public int PersonId { get; set; }

    public int Count { get; set; }

    public int Amount { get; set; }
}

var people = new List<Person>();

var purchasesOfPeople = people.SelectMany(x => x.Purchases).ToList();
//list of Purchases of persons

In [ ]:
string[] fullNames = { "Anne Williams", "John Fred Smith", "Sue Green" };

//we want create list like This
//"Anne", "Williams", "John", "Fred", "Smith", "Sue", Green"

IEnumerable<string> query = fullNames.SelectMany (name => name.Split());

In [ ]:
string[] fullNames = { "Anne Williams", "John Fred Smith", "Sue Green" };

//SelectMany  in query syntax
IEnumerable<string> query =
    from fullName in fullNames
    from name in fullName.Split() // Translates to SelectMany
    select name;

#### Multiple range variables 

The ***extended scope*** variables is the `killer` scenario for `query syntax` over `fluent syntax`.

In [ ]:
string[] fullNames = { "Anne Williams", "Sue Green", "John Fred Smith" };

IEnumerable<string> query =
    from fullName in fullNames
    from name in fullName.Split()
    select name + " came from " + fullName;
/*
Anne came from Anne Williams
Williams came from Anne Williams
John came from John Fred Smith
...
*/

var query1 = 
    from fullName in fullNames
    from name in fullName.Split()
    orderby fullName, name // in this case we lost fullName from outer 
    select name + " came from " + fullName;

foreach(var item in query1)
    Console.WriteLine(item);

var query2 =
    from fullName in fullNames
    from x in fullName.Split().Select (name => new { name, fullName } )
    orderby x.fullName, x.name
    select x.name + " came from " + x.fullName;

foreach(var item in query2)
    Console.WriteLine(item);

In [ ]:
string[] fullNames = { "Anne Williams", "Sue Green", "John Fred Smith" };

//Fluent query
IEnumerable<string> query = fullNames
    .SelectMany (fName => fName.Split()
                            .Select (name => new { name, fName } ))
.OrderBy (x => x.fName)
.ThenBy (x => x.name)
.Select (x => x.name + " came from " + x.fName);

there are two basic patter for use query syntax  
 - ***expanding and flattening subsequences***
 - ***cartesian product, or cross join***

In [ ]:
//expanding and flattening subsequences

string[] fullNames = { "Anne Williams", "Sue Green", "John Fred Smith" };

IEnumerable<string> query =
    from fullName in fullNames
    from name in fullName.Split()
    select name + " came from " + fullName;

//in Ef queries 
IEnumerable<string> query1 = 
    from c in dbContext.Customers
    from p in c.Purchases
    select c.Name + " bought a " + p.Description;

//Here, we’ve expanded each customer into a subsequence of purchases.

In [ ]:
//cartesian product, or cross join

int[] numbers = { 1, 2, 3 }; string[] letters = { "a", "b" };
IEnumerable<string> query = from n in numbers
from l in letters
select n.ToString() + l;
// RESULT: { "1a", "1b", "2a", "2b", "3a", "3b" }

//This style of query is the basis of SelectMany-style joins.

### Joining with SelectMany

In [ ]:
//You can use SelectMany to join two sequences simply by filtering the results of a
//cross product.

string[] players = { "Tom", "Jay", "Mary" };

IEnumerable<string> query = from name1 in players
                            from name2 in players
                            select name1 + " vs " + name2;
                            
//RESULT: { "Tom vs Tom", "Tom vs Jay", "Tom vs Mary",
// "Jay vs Tom", "Jay vs Jay", "Jay vs Mary",
// "Mary vs Tom", "Mary vs "Jay", "Mary vs Mary" }

In [ ]:
string[] players = { "Tom", "Jay", "Mary" };

IEnumerable<string> query = from name1 in players
                            from name2 in players
                            where name1.CompareTo (name2) < 0
                            orderby name1, name2
                            select name1 + " vs " + name2;
                            
//RESULT: { "Jay vs Mary", "Jay vs Tom", "Mary vs Tom" }


### SelectMany in EF Core

***SelectMany*** in **EF Core** can perform `cross joins`,` non-equi joins`, `inner joins`, and `left outer joins`.

In [ ]:
--non-equi join

SELECT 
    Employees.Name, 
    Departments.DepartmentName 
FROM  Employees
JOIN  Departments  ON  Employees.Age > Departments.MinRequiredAge

-- جوین غیر مساوی ،  استفاده از عملگرهای بزرگتر (>), کوچکتر (<), 
-- بزرگتر یا مساوی (>=)، یا کوچکتر یا مساوی (<=) در شروط جوین می‌تواند

<div dir="rtl" style="width:90%; margin:auto">
<h3>استفاده از SelectMany با روابط پیش‌فرض و ad hoc</h3>
<p>
<strong>روابط پیش‌فرض</strong>: این روابط معمولاً در مدل داده‌های اپلیکیشن تعریف شده‌اند. به عنوان مثال، در یک برنامه مدیریت منابع انسانی، ممکن است رابطه‌ای مانند <code>Employee</code> و <code>Department</code> وجود داشته باشد که نشان دهنده این است که هر کارمند به یک بخش تعلق دارد.
</p>

<p><strong>روابط ad hoc</strong>: این روابط بدون تعریف قبلی در مدل داده‌ها برقرار می‌شوند و معمولاً برای پرس‌وجوهای خاص یا تجزیه‌وتحلیل‌های داده‌ای موقت ایجاد می‌شوند. به عنوان مثال، شما ممکن است بخواهید داده‌های مربوط به فروش را با اطلاعات مشتریان بر اساس شرایط خاصی که فقط برای یک گزارش خاص نیاز است، مرتبط کنید.</p>

<h3>تفاوت Select و SelectMany</h3>
<p> اگر یک لیست از کارمندان دارید و هر کارمند لیستی از پروژه‌ها دارد، <code>Select</code> ممکن است برای هر کارمند یک لیست از پروژه‌ها را برگرداند (ساختار درختی)، در حالی که <code>SelectMany</code> تمام پروژه‌ها را در یک لیست واحد و تخت جمع می‌کند، بدون توجه به اینکه به کدام کارمند تعلق دارند.</p>
</div>

In [ ]:
//cross join

var query = from c in dbContext.Customers
            from p in dbContext.Purchases
            select c.Name + " might have bought a " + p.Description;

//matches every customer to every purchase

In [ ]:
//equi-join (base on equel "=" join)

var query = from c in dbContext.Customers
            from p in dbContext.Purchases
            where c.ID == p.CustomerID
            select c.Name + " bought a " + p.Description;

In [ ]:
//If you have collection navigation properties in your entities,
//you dont need where

var query = from c in dbContext.Customers
            from p in c.Purchases
            select c.Name + " bought a " + p.Description;

In [ ]:
//You can add where clauses to such a query for additional filtering.
var query = 
    from c in dbContext.Customers
    where c.Name.StartsWith ("T")
    from p in c.Purchases
    select new { c.Name, p.Description };

// in EF above code equel to below code

var query1 = 
    from c in dbContext.Customers
    from p in c.Purchases
    where c.Name.StartsWith ("T")
    select new { c.Name, p.Description };



for ***local query***, moving the `where` clause `down` would make it `less efficient`. With ***local queries***, you should `filter` **before** `joining`.

In [ ]:
//You can introduce new tables into the mix with additional from clauses.

var query = 
from c in dbContext.Customers
from p in c.Purchases
from pi in p.PurchaseItems
select new { c.Name, p.Description, pi.Detail };

In [ ]:
//To include data from a parent table
//(via a navigation property), you don’t add a from clause

var query = 
from c in dbContext.Customers
select new { Name = c.Name, SalesPerson = c.SalesPerson.Name };

//You don’t use SelectMany in this case because there’s no subcollection to flatten.

### Outer joins with SelectMany

<div dir="rtl" style="width:90%;margin:auto;">
<h2>انواع join ها</h2>
<h3>INNER JOIN</h3>
<ul><li><strong>INNER JOIN</strong>: اتصال داده‌ها بر اساس شرط مشترک بین دو جدول؛ تنها ردیف‌هایی که در هر دو جدول مطابقت دارند، نمایش داده می‌شوند.</li></ul>
<h3>OUTER JOIN</h3>
<ul>
<li><strong>LEFT (OUTER) JOIN</strong>: نمایش تمامی ردیف‌های جدول سمت چپ و ردیف‌های مطابق از جدول سمت راست. ردیف‌هایی از جدول سمت چپ که مطابقت ندارند، با مقادیر <code>NULL</code> برای ستون‌های جدول سمت راست نمایش داده می‌شوند.</li>
<li><strong>RIGHT (OUTER) JOIN</strong>: نمایش تمامی ردیف‌های جدول سمت راست و ردیف‌های مطابق از جدول سمت چپ. ردیف‌هایی از جدول سمت راست که مطابقت ندارند، با مقادیر <code>NULL</code> برای ستون‌های جدول سمت چپ نمایش داده می‌شوند.</li>
<li><strong>FULL (OUTER) JOIN</strong>: ترکیبی از <code>LEFT JOIN</code> و <code>RIGHT JOIN</code> که تمامی ردیف‌ها از هر دو جدول را نمایش می‌دهد. اگر مطابقتی در یکی از جداول وجود نداشته باشد، مقادیر <code>NULL</code> برای ستون‌های جدول مقابل ظاهر می‌شود.</li></ul>

<h3>CROSS JOIN</h3>
<ul><li><strong>CROSS JOIN</strong>: این جوین بدون نیاز به شرط اتصال، هر ردیف از جدول اول را با هر ردیف از جدول دوم ترکیب می‌کند. نتیجه، جدولی است که شامل ترکیبات ممکن هر دو جدول است.</li></ul>

<h3>SELF JOIN</h3>
<ul><li><strong>SELF JOIN</strong>: این نوع جوین برای اتصال یک جدول به خودش به کار می‌رود، معمولاً برای مقایسه ردیف‌ها درون یک جدول با یکدیگر استفاده می‌شود.</li></ul>

<h3>NATURAL JOIN</h3>
<ul><li><strong>NATURAL JOIN</strong>: این نوع اتصال داده‌ها را بر اساس ستون‌های مشترک با نام‌های یکسان در دو جدول ادغام می‌کند، بدون اینکه نیاز به مشخص کردن شرط مطابقت باشد.</li></ul>
</div>

In [ ]:
// normal select 

var normalQuery = 
from c in dbContext.Customers
select new {
            c.Name,
            Purchases = from p in c.Purchases
                        where p.Price > 1000
                        select new { p.Description, p.Price }
};

// this is a outer left join

//selectMany 

var selectManyQuery = 
from c in dbContext.Customers
from p in c.Purchases
where p.Price > 1000
select new { c.Name, p.Description, p.Price };

// but this is a inner join

In [ ]:
//To get a left outer join with a flat result set, 
//we must apply the DefaultIfEmpty query operator on the inner sequence.
var selectManyLeftJoinQuery = 
from c in dbContext.Customers
from p in c.Purchases.DefaultIfEmpty()
select new { c.Name, p.Description, p.Price };

//returning all customers—even if they have no purchases.